In [3]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from os import listdir
from datatable import dt, f, by
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from utils import get_path, data_load

# Descrição

Entre os desafios mais urgentes do Sistema Único de Saúde (SUS) estão a ampliação do acesso e a adequação dos serviços ofertados. No Brasil, as relações existentes entre regiões e redes de atenção à saúde indicam que o avanço do processo de regionalização do SUS pode interferir positivamente na oferta de ações de serviços de saúde, tornando-a mais eficiente e abrangente.

[Saiba mais](https://www.resbr.net.br/apresentacao/)

[Plataforma de acesso aos dados](https://indicadores.resbr.net.br/view/)

# Scrap

In [2]:
def scrap_indicator(driver, url, n_files=10):
  driver.maximize_window()
  anchors = None
  columns = None
  checkboxes = None
  for i in tqdm(range(n_files)):
    driver.get(url + 'clrSession=true')
    driver.find_element(By.XPATH, "//input[@type='radio' and @value='4']").click()
    columns = driver.find_elements(
      By.XPATH, "//input[@type='checkbox' and @onclick='SelAll(this)']")
    for column in columns: column.click()
    driver.find_element(By.XPATH, "//input[@id='btnEnviar']").click()
    if anchors == None:
      anchors = driver.find_elements(By.CSS_SELECTOR, "a[href*=listaLocs]")
      anchors = set([a.get_attribute('href') for a in anchors])
      anchors = sorted(list(anchors))
    for a in anchors: driver.execute_script(a[:-1])
    checkboxes = driver.find_elements(
      By.XPATH, "//input[@type='checkbox' and @onclick='SelAll(this)']")
    n_cks = len(checkboxes)
    pos_list = np.arange(0, n_cks, n_cks/10, dtype=int)
    bgn = pos_list[i]
    try: end = pos_list[i+1]
    except: end = len(checkboxes)
    selected_boxes = checkboxes[bgn:end]
    for checkbox in selected_boxes:
      if not checkbox.is_selected() and checkbox.is_displayed():
        checkbox.click()
    driver.find_element(By.XPATH, "//input[@id='btnEnviar2']").click()
    driver.find_element(By.XPATH, "//input[@id='btnTabela']").click()
    driver.get(url.replace('indicadores.php', 'output.php') + 'export=2')
    print(f'{i+1:02}/{n_files}: checkboxes={n_cks}, selected={len(selected_boxes)}')
    driver.switch_to.window(driver.window_handles[0])

In [3]:
url = 'https://indicadores.resbr.net.br/view/indicadores.php?'
chromedriver = '/Users/lucas/Desktop/chromedriver'
driver = webdriver.Chrome(chromedriver)
scrap_indicator(driver, url)

/var/folders/9d/972tfrtd01z2hhl836z97zdr0000gn/T/ipykernel_83928/4099842114.py:3: DeprecationWarning:

executable_path has been deprecated, please pass in a Service object

 10%|█         | 1/10 [00:22<03:23, 22.56s/it]

01/10: checkboxes=465, selected=46


 20%|██        | 2/10 [00:30<01:49, 13.73s/it]

02/10: checkboxes=465, selected=46


 30%|███       | 3/10 [00:40<01:26, 12.38s/it]

...

08/10: checkboxes=465, selected=46


 90%|█████████ | 9/10 [01:45<00:11, 11.25s/it]

09/10: checkboxes=465, selected=46


100%|██████████| 10/10 [01:56<00:00, 11.68s/it]

10/10: checkboxes=465, selected=51


In [ ]:
# driver.close()

# Tabela resultante

In [8]:
path_dir = get_path('INDICADORES', '')
files = [path_dir+pth for pth in listdir(path_dir) if pth[-3:] == 'csv']
dfs = [pd.read_csv(fl, encoding='latin1', sep=';') for fl in files]
for df in dfs: print(df.shape)

(524, 184)
(595, 184)
(673, 184)
(511, 184)
(645, 184)
(516, 184)
(528, 184)
(695, 184)
(734, 184)
(587, 184)


In [9]:
df_indc = pd.concat(dfs)
df_indc

,CÓDIGO,Tipo Localidade,UF,CIR,Município,Número de municípios na Região de Saúde,Município de capital,Municípios que fazem parte da zona de fronteira - IBGE,Municípios que fazem parte da faixa de fronteira - IBGE,Municípios que fazem parte da fronteira estadual,...,Porcentual de Despesas com Investimentos pelo Total de Despesas em Saúde (2014),Porcentual de Tranferências SUS pelo Total de Despesas em Saúde (2014),Porcentual de Recursos Próprios em Saúde - EC29 (2014),Recursos de Transferências SUS por habitante (2014),Despesas com Recursos Próprios em Saúde por habitante (2014),Cobertura populacional estimada pelas equipes de Atenção Básica (2014),% de população beneficiária de planos de saúde (Setembro/2015),Porcentual de famílias beneficiárias do Programa Bolsa Família (2014),Porcentual de população beneficiária do Programa Bolsa Família (2014),Unnamed: 183
0,15.001,REGSAU,-,Araguaia,-,15,0,0,0,7,...,"8,53","54,61","18,83","213,11","161,66","63,90","5,15","34,43","31,59",NaN
1,15.002,REGSAU,-,Baixo Amazonas,-,14,0,3,5,5,...,"10,61","50,82","20,77","204,22","159,30","42,20","6,18","49,15","52,45",NaN
2,15.003,REGSAU,-,Carajás,-,17,0,0,0,9,...,"9,44","26,16","23,79","177,49","455,16","52,23","22,06","34,18","35,01",NaN
3,15.004,REGSAU,-,Lago de Tucuruí,-,6,0,0,0,0,...,"8,08","53,90","20,08","208,23","166,48","49,99","3,07","45,30","39,86",NaN
4,15.013,REGSAU,-,Marajó I,-,9,0,0,0,0,...,"20,06","51,15","17,21","158,85","111,80","24,19","1,02","60,93","61,59",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
582,171.780,MUNIC,Tocantins,Sudeste,Ponte Alta do Bom Jesus,15,Não,Não,Não,Sim,...,"8,43","34,79","28,54","199,83","369,45",100,"2,02","51,97","46,93",NaN
583,171.800,MUNIC,Tocantins,Sudeste,Porto Alegre do Tocantins,15,Não,Não,Não,Não,...,"10,52","35,89","19,05","220,42","386,41",100,"1,15","52,65","48,82",NaN
584,171.865,MUNIC,Tocantins,Sudeste,Rio da Conceição,15,Não,Não,Não,Sim,...,"13,86","20,14","15,52","162,11","538,36",100,"0,81","47,84","48,19",NaN
585,172.090,MUNIC,Tocantins,Sudeste,Taguatinga,15,Não,Não,Não,Sim,...,"0,25","44,17","33,13","187,31","292,60","78,25","26,20","37,30","41,23",NaN


# Exportando tabela

In [5]:
path_indc = path_dir + 'indicadores.csv.gzip'
df_indc.to_csv(path_indc, index=False)

In [7]:
pd.read_csv(path_indc, low_memory=False)

,CÓDIGO,Tipo Localidade,UF,CIR,Município,Número de municípios na Região de Saúde,Município de capital,Municípios que fazem parte da zona de fronteira - IBGE,Municípios que fazem parte da faixa de fronteira - IBGE,Municípios que fazem parte da fronteira estadual,...,Porcentual de Despesas com Investimentos pelo Total de Despesas em Saúde (2014),Porcentual de Tranferências SUS pelo Total de Despesas em Saúde (2014),Porcentual de Recursos Próprios em Saúde - EC29 (2014),Recursos de Transferências SUS por habitante (2014),Despesas com Recursos Próprios em Saúde por habitante (2014),Cobertura populacional estimada pelas equipes de Atenção Básica (2014),% de população beneficiária de planos de saúde (Setembro/2015),Porcentual de famílias beneficiárias do Programa Bolsa Família (2014),Porcentual de população beneficiária do Programa Bolsa Família (2014),Unnamed: 183
0,15.001,REGSAU,-,Araguaia,-,15,0,0,0,7,...,"8,53","54,61","18,83","213,11","161,66","63,90","5,15","34,43","31,59",NaN
1,15.002,REGSAU,-,Baixo Amazonas,-,14,0,3,5,5,...,"10,61","50,82","20,77","204,22","159,30","42,20","6,18","49,15","52,45",NaN
2,15.003,REGSAU,-,Carajás,-,17,0,0,0,9,...,"9,44","26,16","23,79","177,49","455,16","52,23","22,06","34,18","35,01",NaN
3,15.004,REGSAU,-,Lago de Tucuruí,-,6,0,0,0,0,...,"8,08","53,90","20,08","208,23","166,48","49,99","3,07","45,30","39,86",NaN
4,15.013,REGSAU,-,Marajó I,-,9,0,0,0,0,...,"20,06","51,15","17,21","158,85","111,80","24,19","1,02","60,93","61,59",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6003,171.780,MUNIC,Tocantins,Sudeste,Ponte Alta do Bom Jesus,15,Não,Não,Não,Sim,...,"8,43","34,79","28,54","199,83","369,45",100,"2,02","51,97","46,93",NaN
6004,171.800,MUNIC,Tocantins,Sudeste,Porto Alegre do Tocantins,15,Não,Não,Não,Não,...,"10,52","35,89","19,05","220,42","386,41",100,"1,15","52,65","48,82",NaN
6005,171.865,MUNIC,Tocantins,Sudeste,Rio da Conceição,15,Não,Não,Não,Sim,...,"13,86","20,14","15,52","162,11","538,36",100,"0,81","47,84","48,19",NaN
6006,172.090,MUNIC,Tocantins,Sudeste,Taguatinga,15,Não,Não,Não,Sim,...,"0,25","44,17","33,13","187,31","292,60","78,25","26,20","37,30","41,23",NaN
